In [1]:
#Winning Factor 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

print('Import Done')


Import Done


In [21]:
# drivers = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\DRIVERS.csv")
results = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\RESULTS.csv")
races = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\RACES.csv")
constructors = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\CONSTRUCTORS.csv")
qualifying = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\QUALIFYING.csv")
status = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\STATUS.csv")
pit_stops = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\PIT_STOPS.csv")
lap_times = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\LAP_TIMES.csv")

print('Load Done')

Load Done


In [24]:
# Filter for Mercedes
results_merged = results.merge(constructors, on="CONSTRUCTORID")
mercedes_data = results_merged[results_merged["NAME_y"] == "Mercedes"]

# Join race details
# mercedes_data = mercedes_data.merge(races, on="RACEID", suffixes=("", "_race"))

# # Merge qualifying times
# mercedes_data = mercedes_data.merge(qualifying, on=["RACEID", "DRIVERID"], how="left")

# # Drop unnecessary columns and handle missing
# mercedes_data = mercedes_data.dropna(subset=["Q1", "Q2", "Q3"])


KeyError: 'CONSTRUCTORID'